<a href="https://colab.research.google.com/github/edanurb/Recognition-of-Handwritten-Digits/blob/main/MnistDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from  torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [ ]:
dataset=MNIST(root="data/", download=True,transform=ToTensor())

In [ ]:
def set_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  return torch.device('cpu')


In [ ]:
def to_device(data,device):
  #if isinstance(data, (list,tuple)):
   #     return [to_device(x, device) for x in data]
  return data.to(device,non_blocking=True)

In [ ]:

class DeviceDataLoader():
  def __init__(self,data,device):
    self.data=data
    self.device=device
  
  def __iter__(self):
    for x,y in self.data:
      yield to_device(x,self.device) ,to_device(y,self.device)



In [ ]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size=128

train_set,val_set= random_split(dataset,[50000,10000])
train_loader=DataLoader(train_set,batch_size,shuffle=True,num_workers=2,pin_memory=True)
val_loader=DataLoader(val_set,batch_size,num_workers=2,pin_memory=True)


In [ ]:
device=set_device()

train_loader=DeviceDataLoader(train_loader,device)
val_loader=DeviceDataLoader(val_loader,device)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def accurate(pred,outputs):
  value,preds=torch.max(pred,dim=1)
  return torch.tensor(torch.sum(preds==outputs).item()/len(preds))

class MyModel(torch.nn.Module):
  def __init__(self,hiddenlayer_size,input_size,output_size):
    super().__init__()
    self.hiddenlayer=nn.Linear(input_size,hiddenlayer_size)
    self.outputlayer=nn.Linear(hiddenlayer_size,output_size)

  def forward(self,x):
    #flatten image
    x = x.view(x.size(0), -1)
    #hidden layer outputs
    y_hidden=self.hiddenlayer(x)
    #activation function
    active=F.relu(y_hidden)
    #predictions
    out=self.outputlayer(active)
    return out

  def train_step(self,x,y):
    #generate prediction
    pred= self(x)
    loss=F.cross_entropy(pred,y)
    return loss

  def val_step(self,x,y):
    preds=self(x)
    acc=accurate(preds,y)
    loss=F.cross_entropy(preds,y)
    return {'val_acc': acc.item(),'val_loss': loss.item()}
    


In [ ]:
model=MyModel(36,784,10)
to_device(model,device)


MyModel(
  (hiddenlayer): Linear(in_features=784, out_features=36, bias=True)
  (outputlayer): Linear(in_features=36, out_features=10, bias=True)
)

In [ ]:
def print_acc_loss(epoch,acc,loss):
  print("Epoch[{}]  val_acc:{}  val_loss:{}".format(epoch,sum(acc)/len(acc),sum(loss)/len(loss)))
  

def fit(model,epoch_size,train_laoder,val_loader,lr,opt=torch.optim.SGD):
  optimizer=opt(model.parameters(),lr)
  for epoch in range(epoch_size):
    acc_list=[]
    loss_list=[]
    #train 
    for x,y in train_loader:
      loss=model.train_step(x,y)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    for x,y in val_loader:
      values=model.val_step(x,y)
      acc_list.append(values['val_acc'])
      loss_list.append(values['val_loss'])
    print_acc_loss(epoch,acc_list,loss_list)


In [ ]:
fit(model,5,train_loader,val_loader,0.5)

Epoch[0]  val_acc:0.9591574367088608  val_loss:0.13550962894400465
Epoch[1]  val_acc:0.9618275316455697  val_loss:0.13092784739181965
Epoch[2]  val_acc:0.961629746835443  val_loss:0.12916434513805788
Epoch[3]  val_acc:0.9657832278481012  val_loss:0.11339358360612695
Epoch[4]  val_acc:0.9642998417721519  val_loss:0.11806925390905972


In [ ]:
fit(model,5,train_loader,val_loader,0.1)

Epoch[0]  val_acc:0.9677610759493671  val_loss:0.10784965388099604
Epoch[1]  val_acc:0.9682555379746836  val_loss:0.10368407489378241
Epoch[2]  val_acc:0.9674643987341772  val_loss:0.10414182602227488
Epoch[3]  val_acc:0.9682555379746836  val_loss:0.10463703770331945
Epoch[4]  val_acc:0.9691455696202531  val_loss:0.10375046678170373


In [ ]:
def make_pred(img,model,device):
    img=to_device(img,device)
    _, pred=torch.max(model(img),dim=1)
    return pred.item()

In [ ]:
test_dataset = MNIST(root='data/', 
                     train=False,
                     transform=ToTensor())


for i in range(15):
  img,label=test_dataset[i]
  print("pred:{}   label:{}".format(make_pred(img,model,device),label))

pred:7   label:7
pred:2   label:2
pred:1   label:1
pred:0   label:0
pred:4   label:4
pred:1   label:1
pred:4   label:4
pred:9   label:9
pred:5   label:5
pred:9   label:9
pred:0   label:0
pred:6   label:6
pred:9   label:9
pred:0   label:0
pred:1   label:1
